# input: list of clusters for each vacancy, vacancy text, list of clusters. User's uput - selcted clusters. Output - which vacancies are good

In [1]:
import pickle
import pandas as pd


In [2]:

# clusters for each vacancy
with open('super_skill_list.pkl', 'rb') as f:
    x = pickle.load(f)

hard_cl=[x[i][0] for i in range(len(x))]
soft_cl=[x[i][1] for i in range(len(x))]



In [3]:
s = pd.read_csv("JobDataTranslated.csv")
# s - job describtion
#s['Job Description']

In [4]:
def reduce(l):
    sorted_reduced = []
    for i in l:
        if i not in sorted_reduced:
                sorted_reduced.append(i)

    return sorted_reduced


In [5]:
# list of current clusters
final_skills = pd.read_csv(r"C:\Users\Dima\Desktop\FeatFiles\clusters\final clusters.csv")
hardsk=[]

softsk=list(final_skills["0"].dropna().values)

for col in final_skills.columns[1:]:  
    hardsk+= final_skills[col].dropna().to_list()
    
    
hardsk=list(map(lambda x: x.lower(), hardsk))
hardsk = reduce(hardsk) # hard skills colums 2 and 3 of file

softsk=list(map(lambda x: x.lower(), softsk))
softsk=reduce(softsk) #soft clusters


In [6]:
# this is what user enters
entered_cl_soft = [softsk[3],softsk[1],softsk[4], softsk[16]]
entered_cl_hard = [hardsk[1],hardsk[2],hardsk[3],hardsk[20],hardsk[25]]
print(entered_cl_soft, entered_cl_hard)

['teamwork', 'business management', 'entrepreneurship', 'logical thinking'] ['maintenance', 'repair', 'utilities', 'construction', 'answering phones']


In [7]:
def get_interect(entered, soft=False):
    intersects = []
    l=hard_cl
    if soft:
        l=soft_cl
        
    for i in l:
        if i!=-1:
            intersects.append(list(set(entered) & set(i)))
        else:
            intersects.append(-1)
        
    return intersects


In [8]:

def get_vac(soft=False):
    
    entered_cl=entered_cl_hard
    vac_cl = hard_cl
    if soft:
        entered_cl=entered_cl_soft
        vac_cl = soft_cl
    

    intersects = get_interect(entered_cl, soft)
    
    
    scores_less_challenging = []
    scores_more_challenging=[]

    #for all vacancies
    for i in range(len(vac_cl)):
        have_skills=intersects[i]
       
        
        if have_skills==-1:
            scores_less_challenging.append([-1,i])
            scores_more_challenging.append([-1,i])

        elif len(have_skills)==0:
            scores_less_challenging.append([0,i])
            scores_more_challenging.append([0,i])

        else:
            need_skills = vac_cl[i]

            count=1
            for j in have_skills:
                if need_skills.count(j)!=0:
                    count+=len(need_skills) - need_skills.index(j)+2
            scores_more_challenging.append([count,i])
            #to get less challenging we normalise score (so look at more available vacancies)
            count=count / ((len(need_skills)*(len(need_skills)+1))+0.5)**0.5
            scores_less_challenging.append([count,i])
        
    return scores_less_challenging, scores_more_challenging


In [9]:
def get_output(soft_weight=0.5):
    less,more = get_vac(soft =  False)
    less_soft,more_soft = get_vac(soft =  True)
    
    less.sort(reverse = True)
    more.sort(reverse=True)
    less=less[0:50]
    more = more[0:50]
    
    #less challeging part
    indicies_less = [i[1] for i in less]
    
    final_less = []
    for i in less:#hard part + soft part for the same id
        final_less.append ([i[0]+[ j for j in less_soft if j[1]==i[1] ][0][0]*soft_weight, i[1]])
        
    final_less.sort(reverse=True)
    
    
    final_more = []
    for i in more:#hard part + soft part for the same id
        final_more.append ([i[0]+[ j for j in more_soft if j[1]==i[1] ][0][0]*soft_weight, i[1]])
        
    final_more.sort(reverse=True)
    
    return final_less[:5], final_more[:5]

get_output()

([[3.944932470225567, 271],
  [3.854396724183048, 855],
  [3.732256916886649, 588],
  [3.681608413913329, 878],
  [3.6641361265212504, 834]],
 [[36.5, 271], [36.0, 855], [34.0, 834], [33.0, 826], [33.0, 795]])

In [10]:
def get_best_match(soft_weight=0.5):
    less, more = get_output(soft_weight)
    ind_less=[i[1] for i in less]
    ind_more = [i[1] for i in more]
    both =[i for i in ind_less if (ind_less.count(i)*ind_more.count(i)>0)]
    easier_to_get = [i for i in ind_less if i not in both]
    harder_to_get = [j for j in ind_more if j not in both]
    return both, easier_to_get, harder_to_get
get_best_match(0.4)

([271, 855, 834], [878, 588], [826, 795])

# I think it's nice to have as output first best, then easier, then harder and it will be super cool, if we can allow user to chose weight of soft skills

In [11]:
s['Job Description'][271]

"Rijkhuis BV is looking for an experienced cooperative foreman Do you have several years of practical experience in the (prefab) construction sector and do you want to take responsibility? Do you want to work at a company where you will be involved in building detached luxury homes? Well, then you are the experienced cooperative performer we are looking for. Activities As a cooperating contractor you are involved in the realization of detached luxury homes. The projects are very diverse. It is your task to form a team with the work preparation and the production manager, to ensure that the realization runs as smoothly as possible. As a collaborating contractor you are responsible for the total execution of one or more construction projects and you provide the daily management of all employees on the construction site and subcontractors. Consider the following activities: Coordinating the work on the construction site The timely demand and delivery of materials, equipment and subcontrac